In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+') # tokenizes and removes punctuation
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
import tensorflow
import tflearn
import random
import json
import pickle
import time
from time import sleep 
import pathfinder_nlp as pnlp
import os


chatbot_data = pnlp.load_data()
try:
    with open("chatbot_data.pickle", "rb") as file:
        total_words, titles, x_train, target = pickle.load(file)
except:
    # total_words represents the patterns from the data set after tokenization
    total_words = []
    # title contains all the tags from the data set
    titles = []
    #
    X = []
    #
    Y = []
    for intent in chatbot_data["intents"]:
             for pattern in intent["patterns"]:
                 words =pnlp.sentence_tokenize(pattern)
                 #Appending the tokenized words from our patterns to the total_words list
                 total_words.extend(words)
                 #Tokenized patterns
                 X.append(words)
                 #Tokenized tags
                 Y.append(intent["tag"])
             #Appending the tags to the titles list    
             if intent["tag"] not in titles:
                titles.append(intent["tag"])
    total_words = [pnlp.word_stem(w) for w in total_words if w not in stopwords.words('english')]
    #Sort and remove duplicates in the total_words and titles
    total_words = sorted(list(set(total_words)))
    titles = sorted(titles)

    x_train = []
    target = []#output

    #Initialised the bag with zeros 
    empty_output = [0]*len(titles)
    for x, doc in enumerate(X):
            bag = []
            words = [pnlp.word_stem(w) for w in doc]
            for w in total_words:
                if w in words:
                    bag.append(1)
                else:
                    bag.append(0)
            output = empty_output[:]#output row
            output[titles.index(Y[x])] = 1

            x_train.append(bag)
            target.append(output)

    #Transform x_train and target to an array using numpy.
    x_train = np.array(x_train)
    target = np.array(target)
    #Save total_words, titles, x_train and  target using pickle.
    with open("chatbot_data.pickle", "wb") as file:
        pickle.dump((total_words, titles, x_train, target), file)
tensorflow.compat.v1.reset_default_graph()
neural_network = tflearn.input_data(shape=[None, len(x_train[0])])
neural_network = tflearn.fully_connected(neural_network, 8)
neural_network = tflearn.fully_connected(neural_network, 8)
neural_network = tflearn.fully_connected(neural_network, len(target[0]), activation="softmax")
neural_network = tflearn.regression(neural_network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

model = tflearn.DNN(neural_network)
try:
    if os.path.exists('models/'):
        model.load("models/Pathfinder")
        print("the model has been loaded")
    else:
        raise ValueError
except ValueError:
    #Train the model, n_epoch is the number of itterations.
    #Batch_size is the size of the group the model uses in the training.
    #Show_metrics will show the accuracy and the loss.
    model.fit(x_train, target, n_epoch=1000, batch_size=8, show_metric=True)
    os.makedirs('models/')  
    model.save("models/Pathfinder")#Save the model.
    print('The model has been created and saved')



#The function bag_words will transform the user input to ones and zeros.
def bag_words(sentences, total_words):
    bag_of_words = [0]*len(total_words)
    sentences_total_words = pnlp.sentence_tokenize(sentences)
    sentences_total_words = [pnlp.word_stem(w) for w in sentences_total_words if w not in stopwords.words('english') ]
    for sentence in sentences_total_words:
        for i, w in enumerate(total_words):
            if w == sentence:
                bag_of_words[i] = 1
            
    return np.array(bag_of_words)

#Chat function will take the input from the user and returns response based on propabilities.
def chat(msg):
    bot_name = "Pathfinder"
        #Bag_words function will tranform the user input to bag of words.
    outcomes = model.predict([bag_words(msg, total_words)])
    #Using argmax the model will predict the class.
    outcomes_index = np.argmax(outcomes)
    title = titles[outcomes_index]

    for tag in chatbot_data["intents"]:
        if tag['tag'] == title:
            responses = tag['responses']
            time.sleep(2)
            #The model will choose a random choice from the class predicted.
            results = random.choice(responses)
            return results
                



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Khadi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INFO:tensorflow:Restoring parameters from C:\Users\Khadi\OneDrive\pathfinder modules\models\Pathfinder
the model has been loaded


In [ ]:
import tkinter
from tkinter import *

def send():
    msg=EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    
    if msg!='':
        
        ChatLog.config(state=NORMAL)
        results=chat(msg)
        ChatLog.insert(END,'You:'+ msg +'\n\n') # I changed outcomes to outcomes1
        ChatLog.config(foreground="#442265", font=('Verdana', 12))
        ChatLog.insert(END,'Pathfinder:'+ results +'\n\n') # I changed outcomes to outcomes1
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base=Tk()
base.title('Pathfinder')
base.geometry("450x550") #400x500
base.resizable(width=FALSE,height=FALSE)


#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="10", width=55, font="Arial",)
ChatLog.pack()
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#0000ff", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width=30, height="7", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=375,y=7, height=387)
ChatLog.place(x=7,y=7, height=388, width=375)
EntryBox.place(x=127, y=400, height=85, width=259)
SendButton.place(x=7, y=400, height=84)
base.mainloop()
        